# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [94]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [139]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.feature_importances_ = [0,0,0,0,0]
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.sum(l_c*l_c, axis=1, keepdims=True)/l_s + np.sum(r_c*r_c, axis=1, keepdims=True)/r_s)/(r_s+l_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.nansum(l_c*np.log2(l_c/l_s),axis=1, keepdims=True)+np.nansum(r_c*np.log2(r_c/r_s),axis=1, keepdims=True))/(r_s+l_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1, keepdims=True) + np.max(r_c, axis=1, keepdims=True)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # сортирует сортирует связуку x-y по значениям исков
        # и находит количество уникальных классов в y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # находим индексы в списке sorted_y внутри окна [self.min_samples_split:-self.min_samples_split]
        # где происходит изменение числа. например при sorted_y = [1,1,1,3,3,4] в self.min_samples_split = 2
        # r_border_ids будет равно [3];
        # если в окне только одинаковые классы, то считается, что разбивать по набору признаков х не имеет смысла
        # и возвращается gs = infinity
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        #eq_el_count -- номера элементов на котором происходит изменение числа
        #class_increments -- матрица, строки которой есть количество элементов определённого класса, индекс строки соотве
        #тствует классу, индекс столбца -- порядку изменения числа.
        #Другими словами номера строк матрицы class_increments и порядковые номера r_border_ids или eq_el_count связаны
        #пример: [ 19  22  23  25] - > [  0.  19.   0.] [  3.   0.   0.] [  0.   0.   1.] [  2.   0.   0.]
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)

        # Этот блок нужен чтобы подготовить все варианты разбиения
        # находится кумулятивная сумма class_incremets что есть так же 
        # для каждого порядкового номера разбиения i в  l_class_count[i] количество классов в левом узле дерева
        # r_class_count -- соответственно в правом узле (находится простым вычитанием от изначального распределения классов).
        # в таком случае r_border_ids[i] есть количеттво классов в l_class_count[i]. Далее делается решейп r_borderd_id
        # для дальнейшего удобного броадкаста. 
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # считаются gs функции, которая есть абсолютное значение неопределенности, для 
        # каждого разбиения. Находится разбиене i, при котором достигается минимум неопределённости
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # так как в левом узле должен оказатся признак sorted_x[idx], то нужно поставить границу
        # посередине между sorted_x[idx] и sorted_x[idx+1]. возвращается значение неопределённости и эта граница.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        bincounts = np.bincount(y, minlength=self.num_class)
        probs = bincounts / np.sum(bincounts).astype(float)
        _, counts = np.unique(y, return_counts=True)
        
        if(self.max_depth is not None and depth == self.max_depth) or \
                self.min_samples_split is not None and x.shape[0] <= self.min_samples_split or\
                np.any(np.argmax(counts) / y.size >= self.sufficient_share):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(bincounts), probs)
            return
        
        feature_count = x.shape[1]
        gs_functions = np.zeros(feature_count) # gs fucntions for each feature, ex.: [1,3,1,4,12,8]
        thresholds = np.zeros(feature_count)
        
        min_class = np.min(y)
        y -= min_class
        
        for i in self.get_feature_ids(feature_count):
            gs_functions[i], thresholds[i] = self.__find_threshold(x[:, i], y)
            
        y += min_class
        feature_id = np.argmin(gs_functions)
        
        if np.isnan(thresholds[feature_id]):
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(bincounts), probs)
            return
        
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, thresholds[feature_id])
        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.argmax(bincounts), probs)
            return
  
        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_id, thresholds[feature_id])
        
        if self.G_function == self.__gini:
            I_current = 1 - np.sum((counts / y.size) ** 2)
        elif self.G_function == self.__misclass:
            I_current = (1 - np.max(counts / y.size))
        elif self.G_function == self.__entropy:
            I_current = -np.nansum((counts / y.size) * np.log2(counts / y.size))
            
        self.feature_importances_[feature_id] += (I_current - gs_functions[feature_id])*x.shape[0]

        
        self.__fit_node(x_l, y_l, 2*node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2*node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= sum([(1 - v[0]) for v in self.tree.itervalues()])
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    


In [140]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [141]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [142]:
%time clf.fit(X_train, y_train)

Wall time: 1e+03 µs


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [143]:
%time my_clf.fit(X_train, y_train)

Wall time: 13 ms


## Проверка качества работы на wine

In [144]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.89682539682539686

In [145]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.88571428571428579

## Подготовка данных Speed Dating Data 

In [146]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [159]:
%time clf.fit(X_train, y_train)

Wall time: 70 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [160]:
%time my_clf.fit(X_train, y_train)

Wall time: 998 ms


## Проверка скорости работы на Speed Dating Data 

In [161]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.57057712953022621

In [162]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.57038684027058095

## Проверка качества работы на Speed Dating Data

In [163]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf.fit(X_train, y_train)

Wall time: 65 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [164]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf.fit(X_train, y_train)

Wall time: 999 ms


## Задание 3

In [154]:
from sklearn.pipeline import Pipeline

def prepare():
    global df, X, y
    df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                     encoding='cp1251')
    df = df.iloc[:, :97]
    df = df.drop(['id'], axis=1)
    df = df.drop(['idg'], axis=1)
    df.drop_duplicates(subset=['iid']).gender.value_counts()
    df = df.drop(['condtn'], axis=1)
    df = df.drop(['round'], axis=1)
    df = df.drop(['position', 'positin1'], axis=1)
    df = df.drop(['order'], axis=1)
    df = df.drop(['partner'], axis=1)
    df = df.drop(['age_o', 'race_o', 'pf_o_att',
                  'pf_o_sin', 'pf_o_int',
                  'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
                  'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
                 axis=1)
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = df.drop(['field'], axis=1)
    pd.get_dummies(df, columns=['field_cd'],
                   prefix='field_cd', prefix_sep='=')
    df = df.drop(['undergra'], axis=1)
    df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df = df.dropna(subset=['imprelig', 'imprace'])
    df = df.drop(['from', 'zipcode'], axis=1)
    df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = df.drop(['career'], axis=1)
    df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
                  'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
    df = df.drop(['expnum'], axis=1)
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
        (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                          'temp_totalsum'].T).T * 100
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
        (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                          'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))

        df = df.drop(feat, axis=1)
    df = df.drop(['wave'], axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
        .drop(['gender'], axis=1) \
        .dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
        .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
        .dropna()

    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'),
                           on='pid',
                           how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values

def split():
    global X_train, y_train, X_test, y_test, X, y
    X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.1)



In [155]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
X
y
X_train
X_test
y_train
y_test

pipeline = Pipeline([("prepare", prepare()),
                     ("split", split()),
                     ("classifier", DecisionTreeClassifier(min_samples_split=2))])
pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')

0.5533955076842243

## Задание 4

In [156]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

my_clf = MyDecisionTreeClassifier()
my_clf.fit(X_train, y_train)

features_names = df_pair.columns[1:]
print('DecisionTreeClassifier')
print(pd.Series(index=features_names, data=clf.feature_importances_).sort_values(ascending=False).head(10))
print('MyDecisionTreeClassifier')
print(pd.Series(index=features_names, data=my_clf.feature_importances_).sort_values(ascending=False).head(10))


DecisionTreeClassifier
int_corr      0.067116
field_cd_f    0.034002
fun1_1_f      0.032692
income        0.032090
exphappy      0.030013
attr3_1       0.027378
income_f      0.024830
sinc3_1       0.023442
sinc1_1       0.023389
amb2_1        0.022685
dtype: float64
MyDecisionTreeClassifier
date          0.119831
race          0.099822
attr3_1       0.077143
shar1_1_f     0.068404
fun1_1        0.061171
go_out        0.049140
int_corr      0.045991
age           0.045368
amb3_1_f      0.042915
imprelig_f    0.041018
dtype: float64


## Задание 5

In [157]:
param_dist = {'max_features': ['auto', 'log2', None],
              "min_samples_split": range(2, 12),
              "criterion": ["gini", "entropy"]}

gscv = GridSearchCV(RandomForestClassifier(n_estimators=10), param_dist)
gscv.fit(X_train, y_train)
print(gscv.best_params_)
print(gscv.best_score_)

{'max_features': 'auto', 'min_samples_split': 8, 'criterion': 'gini'}
0.837732703529
